<a href="https://colab.research.google.com/github/Sukanya41455/Tensorflow_practice/blob/main/digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import model_from_json
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train , x_test = x_train.astype('float32')/255, x_test.astype('float32')/255

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# The known number of output classes.
num_classes = 10

# Input image dimensions
img_rows, img_cols = 28, 28

# Channels go last for TensorFlow backend
x_train_reshaped = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test_reshaped = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# Convert class vectors to binary class matrices. This uses 1 hot encoding.
y_train_binary = to_categorical(y_train, num_classes)
y_test_binary = to_categorical(y_test, num_classes)


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

In [5]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# save the model
import os

model_json = model.to_json()

if not os.path.isdir("weights"):
  os.mkdir("weights")
with open("weights/model.json",'w') as f:
  f.write(model_json)

In [7]:
file = "weights/wtimprove{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file, monitor='val_acc',verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

epochs = 2
batch_size = 128

model.fit(x_train_reshaped,y_train_binary,
          batch_size= batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks_list,
          validation_data= (x_test_reshaped,y_test_binary))

Epoch 1/2
469/469 [==============================] - ETA: 0s - loss: 0.2432 - accuracy: 0.9270

469/469 [==============================] - 17s 10ms/step - loss: 0.2432 - accuracy: 0.9270 - val_loss: 0.0507 - val_accuracy: 0.9838
Epoch 2/2
463/469 [============================>.] - ETA: 0s - loss: 0.0833 - accuracy: 0.9753

469/469 [==============================] - 4s 9ms/step - loss: 0.0833 - accuracy: 0.9751 - val_loss: 0.0415 - val_accuracy: 0.9854


In [11]:
print("predict the digit and confidence: ")
import numpy as np
prediction = model.predict(x_test_reshaped[10:20])
print(np.argmax(prediction[1]), max(prediction[1]))

predict the digit and confidence: 
6 0.99996674


In [12]:
import matplotlib.pyplot as plt
%matplotlib inline

# utility function for showing images
def show_imgs(x_test, decoded_imgs=None, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i+1)
        plt.imshow(x_test[i].reshape(28,28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if decoded_imgs is not None:
            ax = plt.subplot(2, n, i+ 1 +n)
            plt.imshow(decoded_imgs[i].reshape(28,28))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.show()